In [1]:
import pandas as pd

Так как файл train.csv весит очень много, то попробуем его обрезать для анализа, а также сразу передавать необходимые типы данных в соответствии с типами данных, заданных в инструкции

In [2]:
dtypes = {
    "row_id": "int64",
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32", 
    "prior_question_had_explanation": "boolean",
}

train = pd.read_csv('riiid-test-answer-prediction/train.csv', low_memory=False, nrows=10**6, dtype=dtypes)

In [3]:
lectures = pd.read_csv('riiid-test-answer-prediction/lectures.csv')
questions = pd.read_csv('riiid-test-answer-prediction/questions.csv')

Для начала проанализируем и сделаем первичную обработку с train

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   row_id                          1000000 non-null  int64  
 1   timestamp                       1000000 non-null  int64  
 2   user_id                         1000000 non-null  int32  
 3   content_id                      1000000 non-null  int16  
 4   content_type_id                 1000000 non-null  boolean
 5   task_container_id               1000000 non-null  int16  
 6   user_answer                     1000000 non-null  int8   
 7   answered_correctly              1000000 non-null  int8   
 8   prior_question_elapsed_time     976277 non-null   float32
 9   prior_question_had_explanation  996184 non-null   boolean
dtypes: boolean(2), float32(1), int16(2), int32(1), int64(2), int8(2)
memory usage: 32.4 MB


In [5]:
train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,False,1,3,1,NaN,<NA>
1,1,56943,115,5716,False,2,2,1,37000.0,False
2,2,118363,115,128,False,0,0,1,55000.0,False
3,3,131167,115,7860,False,3,0,1,19000.0,False
4,4,137965,115,7922,False,4,1,1,11000.0,False


Для начала можно убрать поле row_id, так как оно не несет полезной информации

In [6]:
train = train.drop(['row_id'], axis=1)

Далее можно разделить датасет train на два подсета - с лекциями и с вопросами, а также убрать из них все ненужные поля, а затем уже соединить с таблицами lectures и questions

In [7]:
train_lectures = train[train['content_type_id']]
train_questions = train[~train['content_type_id']]

In [8]:
train_lectures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19907 entries, 89 to 999859
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   timestamp                       19907 non-null  int64  
 1   user_id                         19907 non-null  int32  
 2   content_id                      19907 non-null  int16  
 3   content_type_id                 19907 non-null  boolean
 4   task_container_id               19907 non-null  int16  
 5   user_answer                     19907 non-null  int8   
 6   answered_correctly              19907 non-null  int8   
 7   prior_question_elapsed_time     0 non-null      float32
 8   prior_question_had_explanation  19907 non-null  boolean
dtypes: boolean(2), float32(1), int16(2), int32(1), int64(1), int8(2)
memory usage: 661.0 KB


In [9]:
train_lectures.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
89,653762,2746,6808,True,14,-1,-1,NaN,False
117,10183847,5382,16736,True,21,-1,-1,NaN,False
212,1424348597,5382,30207,True,104,-1,-1,NaN,False
216,1425557777,5382,18545,True,121,-1,-1,NaN,False
295,405813029,8623,10540,True,59,-1,-1,NaN,False


In [10]:
train_lectures = train_lectures.drop(['user_answer', 'answered_correctly', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'content_type_id'], axis=1)

In [11]:
lectures.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [12]:
train_lectures = train_lectures.merge(lectures, left_on='content_id', right_on='lecture_id', how='inner').drop(['lecture_id'], axis=1)

In [13]:
train_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 980093 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   timestamp                       980093 non-null  int64  
 1   user_id                         980093 non-null  int32  
 2   content_id                      980093 non-null  int16  
 3   content_type_id                 980093 non-null  boolean
 4   task_container_id               980093 non-null  int16  
 5   user_answer                     980093 non-null  int8   
 6   answered_correctly              980093 non-null  int8   
 7   prior_question_elapsed_time     976277 non-null  float32
 8   prior_question_had_explanation  976277 non-null  boolean
dtypes: boolean(2), float32(1), int16(2), int32(1), int64(1), int8(2)
memory usage: 31.8 MB


In [14]:
train_questions.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,115,5692,False,1,3,1,NaN,<NA>
1,56943,115,5716,False,2,2,1,37000.0,False
2,118363,115,128,False,0,0,1,55000.0,False
3,131167,115,7860,False,3,0,1,19000.0,False
4,137965,115,7922,False,4,1,1,11000.0,False


In [15]:
train_questions = train_questions.drop(['content_type_id'], axis=1)

In [16]:
questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [17]:
train_questions = train_questions.merge(questions, left_on='content_id', right_on='question_id', how='inner').drop(['question_id'], axis=1)

# Начнем с изучения сета с лекциями

In [18]:
train_lectures.head()

,timestamp,user_id,content_id,task_container_id,tag,part,type_of
0,653762,2746,6808,14,129,2,intention
1,1623661788,24418,6808,121,129,2,intention
2,600004996,91216,6808,312,129,2,intention
3,2461208170,138650,6808,182,129,2,intention
4,10318788851,166728,6808,331,129,2,intention


In [19]:
display(train_lectures.describe())
display(train_lectures.describe(include=['object']))

,timestamp,user_id,content_id,task_container_id,tag,part
count,1.990700e+04,1.990700e+04,19907.000000,19907.000000,19907.000000,19907.000000
mean,7.897040e+09,1.034608e+07,16677.181243,712.355704,90.919425,4.159944
std,1.127003e+10,5.987159e+06,9565.767222,792.733475,54.634111,1.669412
min,2.416820e+05,2.746000e+03,89.000000,2.000000,0.000000,1.000000
25%,9.761835e+08,4.980312e+06,8411.000000,171.000000,47.000000,2.000000
50%,3.618622e+09,9.793549e+06,16363.000000,412.000000,85.000000,5.000000
75%,9.914121e+09,1.556872e+07,24985.000000,960.000000,137.000000,5.000000
max,7.680911e+10,2.093825e+07,32736.000000,5033.000000,187.000000,7.000000


,type_of
count,19907
unique,3
top,concept
freq,14326


In [20]:
train_lectures.task_container_id.value_counts()

17      59
26      51
111     46
56      46
41      46
        ..
4366     1
2327     1
2311     1
2295     1
2023     1
Name: task_container_id, Length: 3010, dtype: int64

task_container_id в таком виде не будет нести какой-либо полезной информации для конечного датасета для обучения. Однако из него можно будет попробовать, например, получить такой признак, как кол-во просмотренных лекций у пользователя в даннном task_container_id

In [21]:
lectures_in_task_container = train_lectures.groupby(['user_id', 'task_container_id'], as_index=False).agg({'content_id': 'count'}).rename({'content_id': 'container_lectures_finished'}, axis=1)

In [22]:
lectures_in_task_container.head()

,user_id,task_container_id,container_lectures_finished
0,2746,14,1
1,5382,21,1
2,5382,104,1
3,5382,121,1
4,8623,59,1


Точно такую же агрегацию сделаю по tag, а затем буду использовать этот признак для того, чтоб отвечать на вопрос, сколько человек лекций вопросов по заданным tags в списке к вопросам

In [23]:
lectures_in_tag = train_lectures.groupby(['user_id', 'tag'], as_index=False).agg({'content_id': 'count'})
lectures_in_tag = lectures_in_tag.rename({'content_id': 'tag_lectures_finished'}, axis=1)

In [24]:
lectures_in_tag.head()

,user_id,tag,tag_lectures_finished
0,2746,129,1
1,5382,40,1
2,5382,43,1
3,5382,58,1
4,8623,30,1


Получим для каждого пользователя такие признаки, как кол-во прослушанных лекцию, timestamp для первой и последней лекции у пользователя и разницу во времени между первой и последней лекцией

In [25]:
def convert_columns(df):
    return ['_'.join(col).rstrip('_') for col in df.columns.values]

In [26]:
user_id_lectures = train_lectures.groupby(['user_id'], as_index=False).agg({'content_id': 'count', 'timestamp': ['min', 'max']})
user_id_lectures.columns = convert_columns(user_id_lectures)
user_id_lectures['timestamp_diff'] = user_id_lectures.timestamp_max - user_id_lectures.timestamp_min

In [27]:
user_id_lectures.head()

,user_id,content_id_count,timestamp_min,timestamp_max,timestamp_diff
0,2746,1,653762,653762,0
1,5382,3,10183847,1425557777,1415373930
2,8623,3,405813029,577424049,171611020
3,12741,6,3083871588,4465486358,1381614770
4,13134,7,12948802833,15906947254,2958144421


Из признака part попробую сделать 7 отдельных признаков, где каждый будет отвечать за то, сколько лекций было послушано в данном part

In [28]:
train_lectures.part.value_counts()

5    9371
2    3819
6    2488
1    1553
3    1002
4     962
7     712
Name: part, dtype: int64

In [29]:
lectures_in_part = train_lectures.groupby(['user_id', 'part'], as_index=False).agg({'content_id': 'count'}).rename({'content_id': 'part_lectures'}, axis=1)

In [30]:
def get_part_user_id(part):
    def get_user_id(user_id):
        global lectures_in_part
        row = lectures_in_part[(lectures_in_part.user_id == user_id) & (lectures_in_part.part == part)]
        
        if len(row) > 0:
            return row.part_lectures.iloc[0]
        
        return 0
    
    return get_user_id

In [31]:
for i in range(1, 8):
    get_lectures_user_id = get_part_user_id(i)
    user_id_lectures[f'lectures_{i}_part'] = user_id_lectures.user_id.apply(get_lectures_user_id)

In [32]:
user_id_lectures = user_id_lectures.rename({
    'content_id_count': 'lectures_amount',
    'timestamp_min': 'lectures_timestamp_min',
    'timestamp_max': 'lectures_timestamp_max',
    'timestamp_diff': 'lectures_timestamp_diff',
}, axis=1)

In [33]:
user_id_lectures.head()

,user_id,lectures_amount,lectures_timestamp_min,lectures_timestamp_max,lectures_timestamp_diff,lectures_1_part,lectures_2_part,lectures_3_part,lectures_4_part,lectures_5_part,lectures_6_part,lectures_7_part
0,2746,1,653762,653762,0,0,1,0,0,0,0,0
1,5382,3,10183847,1425557777,1415373930,1,0,0,0,2,0,0
2,8623,3,405813029,577424049,171611020,2,1,0,0,0,0,0
3,12741,6,3083871588,4465486358,1381614770,0,0,0,3,0,1,2
4,13134,7,12948802833,15906947254,2958144421,1,3,0,0,3,0,0


Таким образом для окончательного датасета я собрал по лекциям 3 таблицы:
- user_id_lectures
- lectures_in_tag
- lectures_in_task_container

# Теперь перейдем к датасету с вопросами

In [34]:
train_questions.head()

,timestamp,user_id,content_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags
0,0,115,5692,1,3,1,NaN,<NA>,5692,3,5,151
1,4216576228,24418,5692,497,3,1,21000.0,True,5692,3,5,151
2,0,51285,5692,0,3,1,NaN,<NA>,5692,3,5,151
3,330907575,91216,5692,153,3,1,22000.0,True,5692,3,5,151
4,560202977,138650,5692,122,3,1,18000.0,True,5692,3,5,151


In [35]:
train_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 980093 entries, 0 to 980092
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   timestamp                       980093 non-null  int64  
 1   user_id                         980093 non-null  int32  
 2   content_id                      980093 non-null  int16  
 3   task_container_id               980093 non-null  int16  
 4   user_answer                     980093 non-null  int8   
 5   answered_correctly              980093 non-null  int8   
 6   prior_question_elapsed_time     976277 non-null  float32
 7   prior_question_had_explanation  976277 non-null  boolean
 8   bundle_id                       980093 non-null  int64  
 9   correct_answer                  980093 non-null  int64  
 10  part                            980093 non-null  int64  
 11  tags                            980093 non-null  object 
dtypes: boolean(1), f

In [36]:
train_questions.describe()

,timestamp,user_id,content_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,bundle_id,correct_answer,part
count,9.800930e+05,9.800930e+05,980093.000000,980093.000000,980093.000000,980093.000000,976277.000000,980093.000000,980093.000000,980093.000000
mean,7.333085e+09,1.016903e+07,5000.238626,808.009877,1.423019,0.650358,25318.599609,4999.823346,1.438840,4.100419
std,1.057230e+10,6.030037e+06,3287.211531,1029.988408,1.156775,0.476857,19703.839844,3287.092819,1.154287,1.674208
min,0.000000e+00,1.150000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,5.169283e+08,4.700718e+06,1999.000000,107.000000,0.000000,0.000000,16000.000000,1997.000000,0.000000,2.000000
50%,2.805786e+09,9.678259e+06,4996.000000,390.000000,1.000000,1.000000,21000.000000,4996.000000,1.000000,5.000000
75%,1.010340e+10,1.556872e+07,7218.000000,1115.000000,3.000000,1.000000,29666.000000,7216.000000,3.000000,5.000000
max,7.809200e+10,2.094902e+07,13522.000000,7739.000000,3.000000,1.000000,300000.000000,13522.000000,3.000000,7.000000


Хотелось бы убрать сразу поля user_answer и correct_answer, так как они кажется не несут какой-то важности, а используются для проверки, что answered_correctly - верный

In [37]:
train_questions = train_questions.drop(['user_answer', 'correct_answer'], axis=1)

Все пропуски заполню 0

In [38]:
train_questions = train_questions.fillna(0)

In [39]:
train_questions.head()

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,part,tags
0,0,115,5692,1,1,0.0,False,5692,5,151
1,4216576228,24418,5692,497,1,21000.0,True,5692,5,151
2,0,51285,5692,0,1,0.0,False,5692,5,151
3,330907575,91216,5692,153,1,22000.0,True,5692,5,151
4,560202977,138650,5692,122,1,18000.0,True,5692,5,151


Создадим признак из bundle_id - сколько в среднем отвечают в нем правильно на вопросы, а также посчитаем std

In [40]:
bundle_id_correct = train_questions.groupby(['bundle_id'], as_index=False).agg({'answered_correctly': ['mean', 'std']})
bundle_id_correct.columns = ['bundle_id', 'mean_correct_bundle', 'std_correct_bundle']

In [41]:
bundle_id_correct.head()

,bundle_id,mean_correct_bundle,std_correct_bundle
0,0,0.863014,0.346212
1,1,0.927273,0.262082
2,2,0.560811,0.496848
3,3,0.798995,0.401763
4,4,0.602606,0.490158


Также поступим с данными по user_id + посчитаем медиану по верным ответам, сколько вопросов человек уже прошел, сколько в среднем раз использовал подсказку

In [42]:
user_id_questions = train_questions.groupby(['user_id'], as_index=False).agg({
    'answered_correctly': ['mean', 'std', 'median', 'count'],
    'prior_question_had_explanation': ['mean'],
})
user_id_questions.columns = ['user_id', 'user_correct_mean', 'user_correct_std', 'user_correct_median', 'user_questions', 'user_explanation_mean',]
user_id_questions.head()

,user_id,user_correct_mean,user_correct_std,user_correct_median,user_questions,user_explanation_mean
0,115,0.695652,0.465215,1.0,46,0.130435
1,124,0.233333,0.430183,0.0,30,0.0
2,2746,0.578947,0.507257,1.0,19,0.578947
3,5382,0.672000,0.471374,1.0,125,0.904
4,8623,0.642202,0.481566,1.0,109,0.880734


Посчитаем долю верных ответов в part у каждого пользователя

In [43]:
part_questions = train_questions.groupby(['user_id', 'part'], as_index=False).agg({
    'answered_correctly': 'mean'
})
part_questions.columns = ['user_id', 'part', 'part_correct']

In [44]:
part_questions.head()

,user_id,part,part_correct
0,115,1,0.702703
1,115,2,1.000000
2,115,3,0.666667
3,115,4,0.333333
4,115,5,1.000000


Таким образом для qustions я собрал следующие таблички:
- part_questions
- user_id_questions
- bundle_id_correct

# Обединение табличек

Начну с объединения всего, что относится к вопросам

In [45]:
part_questions.head()

,user_id,part,part_correct
0,115,1,0.702703
1,115,2,1.000000
2,115,3,0.666667
3,115,4,0.333333
4,115,5,1.000000


In [46]:
user_id_questions.head()

,user_id,user_correct_mean,user_correct_std,user_correct_median,user_questions,user_explanation_mean
0,115,0.695652,0.465215,1.0,46,0.130435
1,124,0.233333,0.430183,0.0,30,0.0
2,2746,0.578947,0.507257,1.0,19,0.578947
3,5382,0.672000,0.471374,1.0,125,0.904
4,8623,0.642202,0.481566,1.0,109,0.880734


In [47]:
bundle_id_correct.head()

,bundle_id,mean_correct_bundle,std_correct_bundle
0,0,0.863014,0.346212
1,1,0.927273,0.262082
2,2,0.560811,0.496848
3,3,0.798995,0.401763
4,4,0.602606,0.490158


In [48]:
train_questions.head()

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,part,tags
0,0,115,5692,1,1,0.0,False,5692,5,151
1,4216576228,24418,5692,497,1,21000.0,True,5692,5,151
2,0,51285,5692,0,1,0.0,False,5692,5,151
3,330907575,91216,5692,153,1,22000.0,True,5692,5,151
4,560202977,138650,5692,122,1,18000.0,True,5692,5,151


In [49]:
train_questions = train_questions.merge(part_questions, on=['part', 'user_id'], how='inner')

In [50]:
train_questions.head()

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,part,tags,part_correct
0,0,115,5692,1,1,0.0,False,5692,5,151,1.000000
1,56943,115,5716,2,1,37000.0,False,5716,5,168,1.000000
2,4216576228,24418,5692,497,1,21000.0,True,5692,5,151,0.664544
3,8270862135,24418,5716,2892,1,30000.0,True,5716,5,168,0.664544
4,140494,24418,4492,8,1,1333.0,False,4492,5,80,0.664544


In [51]:
train_questions = train_questions.merge(user_id_questions, on='user_id', how='inner')

In [52]:
train_questions.head()

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,part,tags,part_correct,user_correct_mean,user_correct_std,user_correct_median,user_questions,user_explanation_mean
0,0,115,5692,1,1,0.0,False,5692,5,151,1.000000,0.695652,0.465215,1.0,46,0.130435
1,56943,115,5716,2,1,37000.0,False,5716,5,168,1.000000,0.695652,0.465215,1.0,46,0.130435
2,118363,115,128,0,1,55000.0,False,128,1,131 149 92,0.702703,0.695652,0.465215,1.0,46,0.130435
3,131167,115,7860,3,1,19000.0,False,7860,1,131 104 81,0.702703,0.695652,0.465215,1.0,46,0.130435
4,137965,115,7922,4,1,11000.0,False,7922,1,131 149 92,0.702703,0.695652,0.465215,1.0,46,0.130435


In [53]:
train_questions = train_questions.merge(bundle_id_correct, on='bundle_id', how='inner')
train_questions = train_questions.drop('bundle_id', axis=1)

In [54]:
train_questions.head()

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,part,tags,part_correct,user_correct_mean,user_correct_std,user_correct_median,user_questions,user_explanation_mean,mean_correct_bundle,std_correct_bundle
0,0,115,5692,1,1,0.0,False,5,151,1.000000,0.695652,0.465215,1.0,46,0.130435,0.721068,0.449141
1,4216576228,24418,5692,497,1,21000.0,True,5,151,0.664544,0.690275,0.462417,1.0,6283,0.994907,0.721068,0.449141
2,0,51285,5692,0,1,0.0,False,5,151,0.400000,0.545455,0.509647,1.0,22,0.545455,0.721068,0.449141
3,330907575,91216,5692,153,1,22000.0,True,5,151,0.611855,0.663415,0.472670,1.0,1845,0.979404,0.721068,0.449141
4,560202977,138650,5692,122,1,18000.0,True,5,151,0.734637,0.713628,0.452274,1.0,1086,0.971455,0.721068,0.449141


In [55]:
train_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 980093 entries, 0 to 980092
Data columns (total 17 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   timestamp                       980093 non-null  int64  
 1   user_id                         980093 non-null  int32  
 2   content_id                      980093 non-null  int16  
 3   task_container_id               980093 non-null  int16  
 4   answered_correctly              980093 non-null  int8   
 5   prior_question_elapsed_time     980093 non-null  float32
 6   prior_question_had_explanation  980093 non-null  boolean
 7   part                            980093 non-null  int64  
 8   tags                            980093 non-null  object 
 9   part_correct                    980093 non-null  float64
 10  user_correct_mean               980093 non-null  float64
 11  user_correct_std                980091 non-null  float64
 12  user_correct_med

Также заполним все пустые ячейки в std_correct_bundle 0.5

In [56]:
train_questions.std_correct_bundle = train_questions.std_correct_bundle.fillna(0.5)

А теперь объединим таблицу получившуюся с данными по лекциям

In [57]:
user_id_lectures.head()

,user_id,lectures_amount,lectures_timestamp_min,lectures_timestamp_max,lectures_timestamp_diff,lectures_1_part,lectures_2_part,lectures_3_part,lectures_4_part,lectures_5_part,lectures_6_part,lectures_7_part
0,2746,1,653762,653762,0,0,1,0,0,0,0,0
1,5382,3,10183847,1425557777,1415373930,1,0,0,0,2,0,0
2,8623,3,405813029,577424049,171611020,2,1,0,0,0,0,0
3,12741,6,3083871588,4465486358,1381614770,0,0,0,3,0,1,2
4,13134,7,12948802833,15906947254,2958144421,1,3,0,0,3,0,0


In [58]:
lectures_in_tag.head()

,user_id,tag,tag_lectures_finished
0,2746,129,1
1,5382,40,1
2,5382,43,1
3,5382,58,1
4,8623,30,1


In [59]:
lectures_in_task_container.head()

,user_id,task_container_id,container_lectures_finished
0,2746,14,1
1,5382,21,1
2,5382,104,1
3,5382,121,1
4,8623,59,1


In [60]:
train_questions = train_questions.merge(user_id_lectures, on='user_id', how='left')

In [61]:
def get_tag_lectures(user_id, tags):
    global lectures_in_tag
    s = 0
    tags_list = list(map(int, tags.split()))
    row = lectures_in_tag[lectures_in_tag.user_id == user_id]
    row = row[(row.tag.isin(tags_list))]
    if len(row) > 0:
        s += row.tag_lectures_finished.sum()
    return s

In [62]:
train_questions['tags_lectures'] = train_questions.apply(lambda x: get_tag_lectures(x.user_id, x.tags), axis=1)

In [63]:
train_questions.head()

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,part,tags,part_correct,...,lectures_timestamp_max,lectures_timestamp_diff,lectures_1_part,lectures_2_part,lectures_3_part,lectures_4_part,lectures_5_part,lectures_6_part,lectures_7_part,tags_lectures
0,0,115,5692,1,1,0.0,False,5,151,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,4216576228,24418,5692,497,1,21000.0,True,5,151,0.664544,...,1.379911e+10,1.311444e+10,21.0,44.0,13.0,7.0,84.0,12.0,0.0,1
2,0,51285,5692,0,1,0.0,False,5,151,0.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,330907575,91216,5692,153,1,22000.0,True,5,151,0.611855,...,1.877577e+10,1.876764e+10,0.0,5.0,0.0,0.0,41.0,6.0,0.0,0
4,560202977,138650,5692,122,1,18000.0,True,5,151,0.734637,...,4.365765e+09,3.806308e+09,3.0,10.0,0.0,0.0,2.0,4.0,0.0,0


In [64]:
check = train_questions.merge(lectures_in_task_container, on=['user_id', 'task_container_id'], how='left')

In [65]:
check[~check.container_lectures_finished.isna()]

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,part,tags,part_correct,...,lectures_timestamp_diff,lectures_1_part,lectures_2_part,lectures_3_part,lectures_4_part,lectures_5_part,lectures_6_part,lectures_7_part,tags_lectures,container_lectures_finished


Кажется, что это объединение ничего не дало, поэтому не будем его добавлять

А теперь заполним все поля новые с nan из лекций 0

In [66]:
train_questions = train_questions.fillna(0)

# Финализация результатов

In [67]:
train_questions.head()

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,part,tags,part_correct,...,lectures_timestamp_max,lectures_timestamp_diff,lectures_1_part,lectures_2_part,lectures_3_part,lectures_4_part,lectures_5_part,lectures_6_part,lectures_7_part,tags_lectures
0,0,115,5692,1,1,0.0,False,5,151,1.000000,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,4216576228,24418,5692,497,1,21000.0,True,5,151,0.664544,...,1.379911e+10,1.311444e+10,21.0,44.0,13.0,7.0,84.0,12.0,0.0,1
2,0,51285,5692,0,1,0.0,False,5,151,0.400000,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,330907575,91216,5692,153,1,22000.0,True,5,151,0.611855,...,1.877577e+10,1.876764e+10,0.0,5.0,0.0,0.0,41.0,6.0,0.0,0
4,560202977,138650,5692,122,1,18000.0,True,5,151,0.734637,...,4.365765e+09,3.806308e+09,3.0,10.0,0.0,0.0,2.0,4.0,0.0,0


In [68]:
train_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 980093 entries, 0 to 980092
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   timestamp                       980093 non-null  int64  
 1   user_id                         980093 non-null  int32  
 2   content_id                      980093 non-null  int16  
 3   task_container_id               980093 non-null  int16  
 4   answered_correctly              980093 non-null  int8   
 5   prior_question_elapsed_time     980093 non-null  float32
 6   prior_question_had_explanation  980093 non-null  boolean
 7   part                            980093 non-null  int64  
 8   tags                            980093 non-null  object 
 9   part_correct                    980093 non-null  float64
 10  user_correct_mean               980093 non-null  float64
 11  user_correct_std                980093 non-null  float64
 12  user_correct_med

Уберем все ненужные поля, которые никак не повлияют на обучение:
- user_id
- content_id
- task_container_id
- tags

А также приведем part к типу category

In [69]:
train_questions = train_questions.drop(['user_id', 'content_id', 'task_container_id', 'tags'], axis=1)

In [70]:
train_questions.part = train_questions.part.astype('category')

In [71]:
train_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 980093 entries, 0 to 980092
Data columns (total 25 columns):
 #   Column                          Non-Null Count   Dtype   
---  ------                          --------------   -----   
 0   timestamp                       980093 non-null  int64   
 1   answered_correctly              980093 non-null  int8    
 2   prior_question_elapsed_time     980093 non-null  float32 
 3   prior_question_had_explanation  980093 non-null  boolean 
 4   part                            980093 non-null  category
 5   part_correct                    980093 non-null  float64 
 6   user_correct_mean               980093 non-null  float64 
 7   user_correct_std                980093 non-null  float64 
 8   user_correct_median             980093 non-null  float64 
 9   user_questions                  980093 non-null  int64   
 10  user_explanation_mean           980093 non-null  Float64 
 11  mean_correct_bundle             980093 non-null  float64 
 12  st

# Датасет для обучения собран
В нем были собраны следующие поля:
- timestamp - время в мс между текущим и самым первым взаимодействием пользователя
- answered_correctly - ответил ли студент верно на вопрос, является целевой переменной
- prior_question_elapsed_time - сколько студент потратил в среднем на ответы на вопросы в прошлом наборе вопросов
- prior_question_had_explanation- использовалась ли подсказка в прошлом наборе вопросов
- part - секция в соответствии с TOEIC test
- part_correct - сколько в среднем пользователь давал верных вопросов в данном part
- user_correct_mean - сколько пользователь в среднем дает верных ответов
- user_correct_std - std верных ответов у пользователя
- user_correct_median - медиана верных ответов у пользователя
- user_questions - на сколько вопросов ответил пользователь
- user_explanation_mean - вероятность использовать подсказку при ответе на вопрос у пользователя
- mean_correct_bundle - вероятность ответить на вопрос в наборе вопросов
- std_correct_bundle - std вероятности ответить на вопрос в наборе вопросов
- lectures_amount - кол-во прослушанных лекций у студента
- lectures_timestamp_min - время первой послушанной лекции
- lectures_timestamp_max - время последней послушанной лекции
- lectures_timestamp_diff - разница во времени между первой и последней лекциями
- lectures_i_part - кол-во послушанных лекций по part == i
- tags_lectures - кол-во послушанных лекций по заданным тегам у вопроса